# PLSDA Cancer Classification on GBM Cell Lines

### Import data from csv

In [1]:
import numpy as np
import csv

In [2]:
csv_filename = 'data.csv'
data = []
with open(csv_filename, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row[1:])
        
data = np.array(data)
X = data[:, 0:-1]
y = data[:, -1]
X = X.astype(int)
y = y.astype(int)

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [4]:
# Print data shape
print("Inputs matrix size: {}".format(X.shape))
print("Outputs matrix size: {}".format(y.shape))
print("\n\n")

NameError: name 'X' is not defined

# PERFORMING PLSDA
Now that we have processed our csv file and correctly stored our oberservations into an array, we will begin to construct our model using PLSDA. After doing some research online we discovered a method for implementing PLSDA via PLSR. We will build a model using PLSR, where our yobs is an array of 0s (mesanchymal) or 1s (proneural). PLSR will make predictions which will result in continous values from 0 to 1. However, we need our data to be discrete so that we can correctly classify each image into a cell type. Thus, following PLSR we will need to threshold our ypred.

In [ ]:
import scipy as sp, numpy as np
import scipy.io as sio
from sklearn.preprocessing import scale, StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneGroupOut, LeaveOneOut
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt